<a href="https://colab.research.google.com/github/teticio/aventuras-con-textos/blob/master/Modelos_generativos_de_texto_de_%C3%BAltima_generaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar librerías

In [4]:
import os
import sys
import json
import random
import subprocess as sp
from IPython.display import clear_output
from IPython.core.display import display, HTML
import numpy as np
import tensorflow as tf
from keras.utils import get_file

checkpoint_dir = os.getcwd().replace('\\', '/') + '/checkpoints'

In [ ]:
sess = tf.Session()

# XLNet

### Instalar modelo

In [0]:
!test -d XLnet-gen || git clone --quiet https://github.com/rusiaaman/XLnet-gen.git
!pip install -q -r XLnet-gen/requirements.txt
!test -e cased_L-24_H-1024_A-16.zip || wget -q https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
!test -d xlnet_cased_L-24_H-1024_A-16 || unzip -q cased_L-24_H-1024_A-16.zip

     |████████████████████████████████| 1.0MB 6.7MB/s 
     |████████████████████████████████| 51kB 19.7MB/s 


### Generar texto

Requiere `pip install tensorflow==1.14.0`

In [0]:
ejemplo = 'Sadly, however, before she could get to a phone to tell anyone- about it, a terribly stupid catastrophe occurred, and the idea was lost forever.'  #@param {type : 'string'}
with open('test', 'wt') as file:
    file.write(ejemplo)
command = [
    sys.executable, 'XLnet-gen/language_generation.py',
    '--model_config_path=xlnet_cased_L-24_H-1024_A-16/xlnet_config.json',
    '--init_checkpoint=xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt',
    '--spiece_model_file=xlnet_cased_L-24_H-1024_A-16/spiece.model',
    '--input_file=test', '--max_mem_length=512', '--num_toks_pred=512',
    '--num_samples=1', '--top_p=0.9'
]
p = sp.Popen(command, stderr=sp.PIPE)
if p.returncode != 0:
    print(p.communicate()[1].decode())
with open('test.xlnet', 'rt') as file:
    for line in file.readlines():
        display(HTML('<p>' + line + '</p>'))

# GPT-2
[Better Language Models and Their Implications](https://openai.com/blog/better-language-models/)

### Instalar el modelo

In [2]:
!test -d gpt-2 || git clone --quiet https://github.com/nshepperd/gpt-2.git
!pip install -q -r gpt-2/requirements.txt

     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 604kB 14.9MB/s 
     |████████████████████████████████| 51kB 23.1MB/s 


### Importar los módulos de GPT-2

In [6]:
if not 'gpt-2' in sys.path:  # hack
    sys.path += ['gpt-2']
if not 'gpt-2/src' in sys.path:  # hack
    sys.path += ['gpt-2/src']
sys.argv = [
    'train.py', '--dataset=../train.txt', '--model_name=345M',
    '--memory_saving_gradients'
]
import train
import model, sample, encoder

W0817 10:29:41.275823 140403767019328 deprecation_wrapper.py:119] From gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0817 10:29:41.290687 140403767019328 deprecation_wrapper.py:119] From gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



### Descargar los pesos del modelo pre-entrenado

In [0]:
if not os.path.exists('gpt-2/models/345M'):
    sp.call([sys.executable, 'download_model.py', '345M'], cwd='gpt-2')

### Definir la función para generar textos

Basado en https://github.com/openai/gpt-2/blob/master/src/interactive_conditional_samples.py

In [0]:
def interact_model(prompt,
                   model_name='117M',
                   seed=None,
                   nsamples=1,
                   batch_size=1,
                   length=None,
                   temperature=1,
                   top_k=0,
                   top_p=0.0):
    """
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
    :top_p=0.0 : Float value controlling diversity. Implements nucleus sampling,
     overriding top_k if set to a value > 0. A good setting is 0.9.
    """
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    cwd = os.getcwd()
    os.chdir(cwd + '/gpt-2')  # hack
    raw_text = prompt
    texts = []

    try:
        enc = encoder.get_encoder(model_name)
        hparams = model.default_hparams()
        with open(os.path.join('models', model_name, 'hparams.json')) as f:
            hparams.override_from_dict(json.load(f))

        if length is None:
            length = hparams.n_ctx // 2
        elif length > hparams.n_ctx:
            raise ValueError("Can't get samples longer than window size: %s" %
                             hparams.n_ctx)

        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(hparams=hparams,
                                        length=length,
                                        context=context,
                                        batch_size=batch_size,
                                        temperature=temperature,
                                        top_k=top_k,
                                        top_p=top_p)

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
        saver.restore(sess, ckpt)

        context_tokens = enc.encode(raw_text)
        for _ in range(nsamples // batch_size):
            out = sess.run(output,
                           feed_dict={
                               context:
                               [context_tokens for _ in range(batch_size)]
                           })[:, len(context_tokens):]
            for i in range(batch_size):
                texts += [enc.decode(out[i])]

    except:
        os.chdir(cwd)  # hack
        raise
    os.chdir(cwd)
    return texts

### Especificar los checkpoints

In [8]:
# modelo pre-entrenado
checkpoint_num = ''
model_checkpoint_path = 'model_checkpoint_path: "model.ckpt"'

with open('gpt-2/models/345M/checkpoint', "wt") as file:
    print(model_checkpoint_path)
    file.write(model_checkpoint_path)
with open('gpt-2/models/345M/counter', "wt") as file:
    file.write(f'{checkpoint_num}')

model_checkpoint_path: "model.ckpt"


### Generar muestras con el modelo

In [0]:
ejemplo = "You're in a desert, walking along in the sand, when all of a sudden you look down and see a tortoise, Leon. It's crawling toward you. You reach down, you flip the tortoise over on its back. The tortoise lays on its back, its belly baking in the hot sun, beating its legs trying to turn itself over, but it can\u2019t, not without your help. But you\u2019re not helping. Why is that?"  #@param {type : 'string'}
numero_de_muestras = 3  #@param {type : 'number'}
temperature = 1  #@param {type : 'number'}
#@markdown La temperatura controla el grado de aleatoriedad (0 = determinista)
top_k = 40  #@param {type : 'integer'}
#@markdown Número de candidatos considerados en el beam search (0 = "greedy", funciona bien con 40)
top_p = 0.9  #@param {type : 'number'}
#@markdown Controla la diversidad. (0 = valor por defecto, funciona bien con 0.9)
texts = interact_model(prompt=ejemplo,
                       model_name='345M',
                       nsamples=numero_de_muestras,
                       temperature=temperature,
                       top_k=top_k,
                       top_p=top_p)
for i, text in enumerate(texts):
    display(
        HTML('<p>' + "=" * 40 + " SAMPLE " + str(i + 1) + " " + "=" * 40 +
             '</p>'))
    display(
        HTML('<p>' +
             ('<b><i>' + ejemplo + '</b></i>' + text).replace('\n', '<br>') +
             '</p>'))
    display(HTML('<p>' + "=" * 80 + '</p>'))
    display(HTML('<p><br></p>'))

https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf

### Contestar una pregunta sobre un texto

In [0]:
contexto = 'The 2008 Summer Olympics torch relay was run from March 24 until August 8, 2008, prior to the 2008 Summer \n\
Olympics, with the theme of “one world, one dream”. Plans for the relay were announced on April 26, 2007, in \n\
Beijing, China. The relay, also called by the organizers as the “Journey of Harmony”, lasted 129 days and carried \n\
the torch 137,000 km (85,000 mi) – the longest distance of any Olympic torch relay since the tradition was started \n\
ahead of the 1936 Summer Olympics. \n\
\n\
After being lit at the birthplace of the Olympic Games in Olympia, Greece on March 24, the torch traveled to the \n\
Panathinaiko Stadium in Athens, and then to Beijing, arriving on March 31. From Beijing, the torch was \n\
following a route passing through six continents. The torch has visited cities along the Silk Road, symbolizing \n\
ancient links between China and the rest of the world. The relay also included an ascent with the flame to the top of \n\
Mount Everest on the border of Nepal and Tibet, China from the Chinese side, which was closed specially for the \n\
event. \n\
\n\
Q: What was the theme \n\
A: “one world, one dream”. \n\
Q: What was the length of the race? \n\
A: 137,000 km \n\
Q: Was it larger than previous ones? \n\
A: No \n\
Q: Where did the race begin? \n\
A: Olympia, Greece \n\
Q: Is there anything notable about that place? \n\
A: birthplace of Olympic Games \n\
Q: Where did they go after? \n\
A: Athens \n\
Q: How many days was the race? \n\
A: seven \n\
Q: Did they visit any notable landmarks? \n\
A: Panathinaiko Stadium \n'

pregunta = 'Q: And did they climb any mountains? \n\
A:'

display(
    HTML('<p>' +
         (contexto + '<b><i>' + pregunta + '</b></i>').replace('\n', '<br>') +
         '</p>'))
texts = interact_model(prompt=contexto + pregunta,
                       model_name='345M',
                       temperature=0.001)
answer = texts[0][:texts[0].find('Q')]
display(HTML('<p>' + answer + '</p>'))

In [0]:
contexto = 'Tom goes everywhere with Catherine Green, a 54-year-old secretary. He moves around her office at work and goes \n\
shopping with her. ”Most people don’t seem to mind Tom,” says Catherine, who thinks he is wonderful. ”He’s my \n\
fourth child,” she says. She may think of him and treat him that way as her son. He moves around buying his food, \n\
paying his health bills and his taxes, but in fact Tom is a dog. \n\
\n\
Catherine and Tom live in Sweden, a country where everyone is expected to lead an orderly life according to rules \n\
laid down by the government, which also provides a high level of care for its people. This level of care \n\
costs money. \n\
\n\
People in Sweden pay taxes on everything, so aren’t surprised to find that owning a dog means more \n\
taxes. Some people are paying as much as 500 Swedish kronor in taxes a year for the right to keep their dog, which \n\
is spent by the government on dog hospitals and sometimes medical treatment for a dog that falls ill. However, most \n\
such treatment is expensive, so owners often decide to offer health and even life for their dog. \n\
\n\
In Sweden dog owners must pay for any damage their dog does. A Swedish Kennel Club official explains what this means: \n\
if your dog runs out on the road and gets hit by a passing car, you, as the owner, have to pay \n\
for any damage done to the car, even if your dog has been killed in the accident. \n\
\n\
Q: How old is Catherine? \n\
A: 54 \n'

pregunta = 'Q: where does she live? \n\
A:'

display(
    HTML('<p>' +
         (contexto + '<b><i>' + pregunta + '</b></i>').replace('\n', '<br>') +
         '</p>'))
texts = interact_model(prompt=contexto + pregunta,
                       model_name='345M',
                       temperature=0.001)
answer = texts[0][:texts[0].find('Q')]
display(HTML('<p>' + answer + '</p>'))

### Resumir un texto

In [0]:
contexto = '1. Introduction \n\
\n\
Machine learning systems now excel (in expectation) at \n\
tasks they are trained for by using a combination of large \n\
datasets, high-capacity models, and supervised learning \n\
(Krizhevsky et al., 2012) (Sutskever et al., 2014) (Amodei \n\
et al., 2016). Yet these systems are brittle and sensitive to \n\
slight changes in the data distribution (Recht et al., 2018) \n\
and task specification (Kirkpatrick et al., 2017). Current \n\
systems are better characterized as narrow experts rather than \n\
competent generalists. We would like to move towards more \n\
general systems which can perform many tasks – eventually \n\
without the need to manually create and label a training \n\
dataset for each one. \n\
\n\
The dominant approach to creating ML systems is to collect a \n\
dataset of training examples demonstrating correct \n\
behavior for a desired task, train a system to imitate these \n\
behaviors, and then test its performance on independent \n\
and identically distributed (IID) held-out examples. This \n\
has served well to make progress on narrow experts. But \n\
the often erratic behavior of captioning models (Lake et al., \n\
2017), reading comprehension systems (Jia & Liang, 2017), \n\
and image classifiers (Alcorn et al., 2018) on the diversity \n\
and variety of possible inputs highlights some of the shortcomings \n\
of this approach. \n\
\n\
Our suspicion is that the prevalence of single task training \n\
on single domain datasets is a major contributor to the lack \n\
of generalization observed in current systems. Progress \n\
towards robust systems with current architectures is likely \n\
to require training and measuring performance on a wide \n\
range of domains and tasks. Recently, several benchmarks \n\
have been proposed such as GLUE (Wang et al., 2018) and \n\
decaNLP (McCann et al., 2018) to begin studying this. \n\
\n\
Multitask learning (Caruana, 1997) is a promising framework for \n\
improving general performance. However, multitask training in NLP \n\
is still nascent. Recent work reports modest performance \n\
improvements (Yogatama et al., \n\
2019) and the two most ambitious efforts to date have \n\
trained on a total of 10 and 17 (dataset, objective) \n\
pairs respectively (McCann et al., 2018) (Bowman et al., \n\
2018). From a meta-learning perspective, each (dataset, \n\
objective) pair is a single training example sampled \n\
from the distribution of datasets and objectives. Current \n\
ML systems need hundreds to thousands of examples to \n\
induce functions which generalize well. This suggests that \n\
multitask training many need just as many effective training \n\
pairs to realize its promise with current approaches. It will \n\
be very difficult to continue to scale the creation of datasets \n\
and the design of objectives to the degree that may be required  \n\
to brute force our way there with current techniques. \n\
This motivates exploring additional setups for performing \n\
multitask learning. \n'

pregunta = 'TL;DR:'  # Too Long; Didn't Read (¡resúmelo por favor!)

display(
    HTML('<p>' +
         (contexto + '<b><i>' + pregunta + '</b></i>').replace('\n', '<br>') +
         '</p>'))
texts = interact_model(prompt=contexto + pregunta,
                       model_name='345M',
                       temperature=0.001)
answer = texts[0][:texts[0].find('.') + 1]
display(HTML('<p>' + answer + '</p>'))

### Descargar un texto para fine-tunear el modelo



In [2]:
if os.path.exists('./train.txt'):
    os.remove('./train.txt')

# The Hitchhikers Guide to the Galaxy
get_file(
    os.getcwd() + '/train.txt',
    origin=
    'https://docs.google.com/uc?export=download&id=1yQgoz5QmvbxQzN2TZWWjvniPqDhGoC8W'
)

# Lord of the Rings : The Fellowship of the Ring
# get_file(
#     os.getcwd() + '/train.txt',
#     origin=
#     'https://docs.google.com/uc?export=download&id=1kZOHpaPV2ml8rT9l_bbWPt06H6qz-kcm'
# )

# Game of Thrones: A Song of Fire and Ice
# get_file(
#     os.getcwd() + '/train.txt',
#     origin=
#     'https://docs.google.com/uc?export=download&id=1saZmZA07QAqkG-afxlHDPEXNhDNV8Qka'
# )

# Cien Años de Soledad (no funciona muy bien al no haber sido entrenado en español el modelo...)
# get_file(
#     os.getcwd() + '/train.txt',
#     origin=
#     'https://docs.google.com/uc?export=download&id=1XDJmGP9MtOLQSujO5Voicp1wyekHchpm'
# )

# Harry Potter and the Goblet of Fire
# get_file(
#     os.getcwd() + '/train.txt',
#     origin=
#     'https://docs.google.com/uc?export=download&id=10OhbIQHNJrtBiKer8tP_LbxjASqItNzZ'
# )

# Trump's pearls of wisdom (http://www.trumptwitterarchive.com/archive)
# get_file(
#     os.getcwd() + '/train.txt',
#     origin=
#     'https://docs.google.com/uc?export=download&id=1d7g95QNkpvC4bwy5xtsC33JYb-AxvJ-q'
# )

encoding = 'cp1252' if os.name == 'nt' else 'utf-8'
raw_text = ''
with open('train.txt', 'r', encoding=encoding, errors='backslashreplace') as f:
    raw_text += f.read()
with open('train.txt', 'w', encoding=encoding, errors='backslashreplace') as f:
    f.write(raw_text)

278528/271286 [==============================] - 0s 1us/step


### Crear directorio para guardar los checkpoints (si estamos en Colab)

In [6]:
try:  # estamos en Google Colab?
    from google.colab import drive
    drive.mount('/content/drive')
    checkpoint_dir = '/content/drive/My Drive/Colab Notebooks/checkpoints'
except:
    pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Especificar los checkpoints iniciales

In [8]:
# modelo pre-entrenado
checkpoint_num = ''
model_checkpoint_path = 'model_checkpoint_path: "model.ckpt"'

with open('gpt-2/models/345M/checkpoint', "wt") as file:
    print(model_checkpoint_path)
    file.write(model_checkpoint_path)
with open('gpt-2/models/345M/counter', "wt") as file:
    file.write(f'{checkpoint_num}')

model_checkpoint_path: "model.ckpt"


### Entrenar el modelo

https://github.com/nshepperd/gpt-2.git

In [9]:
train.CHECKPOINT_DIR = checkpoint_dir
tf.reset_default_graph()
cwd = os.getcwd()
os.chdir(cwd + '/gpt-2')  # hack
try:
    train.main()
except:
    os.chdir(cwd)  # hack
    raise
os.chdir(cwd)  # hack

Loading checkpoint models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  2.27it/s]


dataset has 64325 tokens
Training...
interrupted
Saving /home/teticio/ML/aventuras-con-textos/checkpoints/run1/model-1


### Especificar los checkpoints

In [0]:
# modelo tuneado con Harry Potter and the Goblet of Fire
checkpoint = checkpoint_dir + '/run1_Harry_Potter'
checkpoint_num = '16000'
model_checkpoint_path = 'model_checkpoint_path: "' + checkpoint + '/model-' + checkpoint_num + '"'

# modelo tuneado con Game of Thrones: A Song of Fire and Ice
#checkpoint = checkpoint_dir + '/run1_GOT'
#checkpoint_num = '18754'
#model_checkpoint_path = 'model_checkpoint_path: "' + checkpoint + '/model-' + checkpoint_num + '"'

with open('gpt-2/models/345M/checkpoint', "wt") as file:
    print(model_checkpoint_path)
    file.write(model_checkpoint_path)
with open('gpt-2/models/345M/counter', "wt") as file:
    file.write(f'{checkpoint_num}')

model_checkpoint_path: "/content/drive/My Drive/Colab Notebooks/checkpoints/run1_Harry_Potter/model-16000"


### Generar muestras con el modelo

In [0]:
ejemplo = 'CHAPTER 1 - The Return of Voldermort'  #@param {type : 'string'}
numero_de_muestras = 3  #@param {type : 'number'}
temperature = 1  #@param {type : 'number'}
#@markdown La temperatura controla el grado de aleatoriedad (0 = determinista)
top_k = 40  #@param {type : 'integer'}
#@markdown Número de candidatos considerados en el beam search (0 = "greedy", funciona bien con 40)
top_p = 0.9  #@param {type : 'number'}
#@markdown Controla la diversidad. (0 = valor por defecto, funciona bien con 0.9)
texts = interact_model(prompt=ejemplo,
                       model_name='345M',
                       nsamples=numero_de_muestras,
                       temperature=temperature,
                       top_k=top_k,
                       top_p=top_p)
for i, text in enumerate(texts):
    display(
        HTML('<p>' + "=" * 40 + " SAMPLE " + str(i + 1) + " " + "=" * 40 +
             '</p>'))
    display(
        HTML('<p>' +
             ('<b><i>' + ejemplo + '</b></i>' + text).replace('\n', '<br>') +
             '</p>'))
    display(HTML('<p>' + "=" * 80 + '</p>'))
    display(HTML('<p><br></p>'))

### Traducir una frase

In [0]:
# puedes encontrar otros texto paralelos aquí
# cuidado porque se actualizan periodicamente
# https://www.manythings.org/anki/

path_to_zip = get_file(
    'spa-eng.zip',
    origin=
    'http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)
path_to_file = os.path.dirname(path_to_zip) + "/spa-eng/spa.txt"

frases = []
with open(path_to_file, 'rt') as file:
    for line in file.readlines():
        tab = line.find('\t')
        frases += [line[:tab] + ' = ' + line[tab + 1:]]
with open('train.txt', 'wt') as file:
    for _ in np.random.choice(range(len(frases)), len(frases), replace=False):
        file.write(frases[_])

### Especificar los checkpoints iniciales

In [0]:
# modelo pre-entrenado
checkpoint_num = ''
model_checkpoint_path = 'model_checkpoint_path: "model.ckpt"'

with open('gpt-2/models/345M/checkpoint', "wt") as file:
    print(model_checkpoint_path)
    file.write(model_checkpoint_path)
with open('gpt-2/models/345M/counter', "wt") as file:
    file.write(f'{checkpoint_num}')

model_checkpoint_path: "model.ckpt"


### Entrenar el modelo

In [0]:
train.CHECKPOINT_DIR = checkpoint_dir
tf.reset_default_graph()
cwd = os.getcwd()
os.chdir(cwd + '/gpt-2')  # hack
try:
    train.main()
except:
    os.chdir(cwd)  # hack
    raise
os.chdir(cwd)  # hack

### Especificar los checkpoints

In [0]:
# modelo tuneado con traducciones de inglés a español
checkpoint = checkpoint_dir + '/run1_spa-eng'
checkpoint_num = '45000'
model_checkpoint_path = 'model_checkpoint_path: "' + checkpoint + '/model-' + checkpoint_num + '"'

with open('gpt-2/models/345M/checkpoint', "wt") as file:
    print(model_checkpoint_path)
    file.write(model_checkpoint_path)
with open('gpt-2/models/345M/counter', "wt") as file:
    file.write(f'{checkpoint_num}')

model_checkpoint_path: "/home/teticio/ML/aventuras-con-textos/checkpoints/run1_spa-eng/model-45000"


In [0]:
ejemplo = "Who did you go to the movies with? ="  #@param {type : 'string'}
numero_de_muestras = 1  #@param {type : 'number'}
temperature = 0.1  #@param {type : 'number'}
#@markdown La temperatura controla el grado de aleatoriedad (0 = determinista)
top_k = 40  #@param {type : 'integer'}
#@markdown Número de candidatos considerados en el beam search (0 = "greedy", funciona bien con 40)
top_p = 0.1  #@param {type : 'number'}
#@markdown Controla la diversidad. (0 = valor por defecto, funciona bien con 0.9)
texts = interact_model(prompt=ejemplo,
                       model_name='345M',
                       nsamples=numero_de_muestras,
                       temperature=temperature,
                       top_k=top_k,
                       top_p=top_p)
display(HTML('<p><b><i>' + ejemplo + '</b></i></p>'))
display(HTML('<p>' + texts[0][:texts[0].find('.') + 1] + '</p>'))

### Trumpy

In [7]:
# modelo tuneado con los tweets de Trump
checkpoint = checkpoint_dir + '/run1_Trump'
checkpoint_num = '22000'
model_checkpoint_path = 'model_checkpoint_path: "' + checkpoint + '/model-' + checkpoint_num + '"'

with open('gpt-2/models/345M/checkpoint', "wt") as file:
    print(model_checkpoint_path)
    file.write(model_checkpoint_path)
with open('gpt-2/models/345M/counter', "wt") as file:
    file.write(f'{checkpoint_num}')

get_file(
    os.getcwd() + '/train.txt',
    origin=
    'https://docs.google.com/uc?export=download&id=1d7g95QNkpvC4bwy5xtsC33JYb-AxvJ-q'
)
encoding = 'cp1252' if os.name == 'nt' else 'utf-8'
with open('train.txt', 'r', encoding=encoding, errors='backslashreplace') as f:
    words = [
        ' '.join(line.split()[0:2]) for line in f.readlines()
        if len(line.split()) >= 2
    ]

model_checkpoint_path: "/content/drive/My Drive/Colab Notebooks/checkpoints/run1_Trump/model-22000"
3121152/Unknown - 1s 0us/step

In [16]:
numero_de_muestras = 10  #@param {type : 'number'}
temperature = 1  #@param {type : 'number'}
#@markdown La temperatura controla el grado de aleatoriedad (0 = determinista)
top_k = 40  #@param {type : 'integer'}
#@markdown Número de candidatos considerados en el beam search (0 = "greedy", funciona bien con 40)
top_p = 0.9  #@param {type : 'number'}
#@markdown Controla la diversidad. (0 = valor por defecto, funciona bien con 0.9)
texts = []
for _ in range(numero_de_muestras):
    ejemplo = random.choice(words)
    text = ejemplo + ' ' + interact_model(prompt=ejemplo,
                                          model_name='345M',
                                          nsamples=1,
                                          temperature=temperature,
                                          top_k=top_k,
                                          top_p=top_p)[0]
    display(
        HTML(
            '<p><img src="https://pbs.twimg.com/profile_images/874276197357596672/kUuht00m_400x400.jpg" width=48 align=left></img><b>Donald J. Trump</b> @realDonaldTrumpy<br>'
            + text[:text.find('\n')] + '</p>'))
    print()